In [1]:
import pyodbc
import pandas as pd

In [2]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = r'DESKTOP-L3GBMQ5\SQLEXPRESS'
DATABASE_NAME = 'Healthcare_Case_Study'

connection_string = (
    f"DRIVER={{{DRIVER_NAME}}};"
    f"SERVER={SERVER_NAME};"
    f"DATABASE={DATABASE_NAME};"
    f"Trusted_Connection=yes;"
)

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
print("Connected successfully!")

Connected successfully!


Departments with the Longest Average Stays

In [3]:
cursor.execute("""
            SELECT
                Department
                ,ROUND(AVG(Length_of_Stay), 2) AS Average_Length_of_Stay
            FROM
                Hospital_Operations_Cleaned_2
            GROUP BY
                Department
            ORDER BY
                Average_Length_of_Stay DESC;
               """)

In [4]:
conn.commit()

Region with the Highest Readmission Rate

In [8]:
cursor.execute("""
            SELECT
                Region
                ,ROUND(SUM( 
                    CAST(Readmission_30Days AS FLOAT)) * 100.0 / COUNT(*), 2) AS Readmission_Rate_Percent
                FROM
                    Hospital_Operations_Cleaned_2
                GROUP BY
                    Region
                ORDER BY
                    Readmission_Rate_Percent DESC;
               """)

In [9]:
conn.commit()

Insurance Type Contributing the Most to Revenue

In [10]:
cursor.execute("""
            SELECT
                Insurance_Type
                ,ROUND(SUM(Total_Revenue), 2) AS Total_Revenue
                ,ROUND(SUM(Total_Revenue) * 001.0 / SUM(SUM(Total_Revenue)) OVER (), 2) AS Revenue_Percent
            FROM
                Hospital_Operations_Cleaned_2
            GROUP BY
                Insurance_Type
            ORDER BY
                Total_Revenue DESC;
            """)

In [11]:
conn.commit()

Correlation between Wait and Satisfaction

In [17]:
cursor.execute("""
WITH Stats AS (
    SELECT 
        AVG(Wait_Time_Minutes) AS Avg_Wait,
        AVG(Satisfaction_Score) AS Avg_Satisfaction,
        STDEV(Wait_Time_Minutes) AS Std_Wait,
        STDEV(Satisfaction_Score) AS Std_Satisfaction,
        COUNT(*) AS N
    FROM Hospital_Operations_Cleaned_2
)
SELECT 
    ROUND(
        (SELECT SUM((h.Wait_Time_Minutes - s.Avg_Wait) * (h.Satisfaction_Score - s.Avg_Satisfaction))
         FROM Hospital_Operations_Cleaned_2 h, Stats s)
        /
        ((SELECT (s.N - 1) * s.Std_Wait * s.Std_Satisfaction FROM Stats s)),
        3
    ) AS Correlation_Wait_Satisfaction;
                """)

In [18]:
conn.commit()